# 1. Site-specific Ligand Binding

## Computing the states with binary vectors

Given a macromolecule with multiple, potentially interacting, ligand binding sites we would like to represent these states in a systematic way.

Firstly we look at how many different *combinations* of bound ligand can arise. This gives some idea of the potential complexity of the system.

**Use the slider below to toggle the number of sites (**n_sites**):**

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import scipy.special

def num_lig_states(n_sites):
    print('The total number of liganded states for',
          n_sites,'sites is: ',2**n_sites,'\n')
    for i in range(n_sites+1):
        print('The number of states with',i,
              'ligands bound is:',int(scipy.special.binom(n_sites, i)))

interactive(num_lig_states,n_sites=(1,10,1))

interactive(children=(IntSlider(value=5, description='n_sites', max=10, min=1), Output()), _dom_classes=('widg…

To uniquely represent the liganded states, it is intuitive to use a binary vector (or string), where $0$ represents an unoccupied site and $1$ represents an occupied sites. 

The binary strings for all liganded states, grouped by stoichiometry, is computed below.

In [3]:
import itertools

def represent_states(n_sites):
    all_states = [''.join(seq) for seq in 
                  itertools.product('01', repeat=n_sites)]
    
    grouped_states = [[] for i in range(n_sites+1)] # initialize list

    for state in all_states:                        # count number
        n_ligands = state.count('1')                # of occupied sites
        grouped_states[n_ligands].append(state)
        
    [i.reverse() for i in grouped_states]           # swap order

    for i,states in enumerate(grouped_states):      # print state vectors
        print('States with',i,'ligands bound:')
        print(states,'\n')
    
interactive(represent_states,n_sites=(1,10,1))

interactive(children=(IntSlider(value=5, description='n_sites', max=10, min=1), Output()), _dom_classes=('widg…

## Cyclic symmetry
One of the advantages of binary representation is that it is straightforward to apply symmetry. For example a cyclic protein oligomer with one ligand binding site per subunit. Two states are identical if their binary representations are equivalent under cyclic permutation.

This is done computationally below (using a brute force approach):

In [4]:
from pprint import pprint

# function that applies cyclic symmetry to a binary vector
def cycle_elements(l, n):
    '''Cycle the elements of a list, l, from right to left by n spaces'''
    return l[n:] + l[:n]

def represent_cyclic_states(n_sites):

    all_states = [''.join(seq) for seq in 
                  itertools.product('01', repeat=n_sites)]
    
    grouped_states = [[] for i in range(n_sites+1)] # initialize list

    for state in all_states:                        # count number
        n_ligands = state.count('1')                # of occupied sites
        grouped_states[n_ligands].append(state)
        
    [i.reverse() for i in grouped_states]           # swap order
    
    cyclic_states = [[] for i in range(n_sites+1)]
    
    for i,states in enumerate(grouped_states):
        
        # create a copy as the base for a cyclic list
        dup_states = states.copy()
        
        # cycle through states with the same number of ligands
        # bound until all states have been accounted for
        while(dup_states != []):
            degen = 1
            k = 1
            perm_state = []
            
            # cycle through symmetry-equivalent states 
            # until returning to the start
            while(dup_states[0] != perm_state):
                perm_state = cycle_elements(dup_states[0],k)
                
                # if the cyclic permutation is within the list of states
                # account for it by removing it and adding degeneracy
                if(perm_state in dup_states[1:]):
                    degen = degen + 1
                    dup_states.remove(perm_state)
                k = k + 1
            
            # add the state that is to be consider the non-redundant state
            # to the cyclic states list and remove it from the original list
            cyclic_states[i].append([dup_states[0],degen])
            del(dup_states[0])
    pprint(cyclic_states)

interactive(represent_cyclic_states,n_sites=(1,10,1))

interactive(children=(IntSlider(value=5, description='n_sites', max=10, min=1), Output()), _dom_classes=('widg…

The non-redundant vector and degeneracy (number of equivalent states) for each state is given in the output above.